In [1]:
import numpy as np
from numba import njit

import jitr

In [2]:
nchannels = 2
nbasis = 40

sys = jitr.ProjectileTargetSystem(
    2 * np.pi * 3 * np.ones(nchannels),
    np.arange(0, nchannels, dtype=np.int32),
    mass_target=44657,
    mass_projectile=938.3,
    Ztarget=20,
    Zproj=1,
    nchannels=nchannels,
)

In [3]:
channels = sys.build_channels_kinematics(E_lab=42.1)

In [4]:
solver = jitr.RMatrixSolver(nbasis)

In [5]:
free_matrices = solver.free_matrix(sys.channel_radii, sys.l, full_matrix=False)

In [6]:
@njit
def potential(r, depth):
    return depth * np.exp(-r / 4)

In [7]:
interaction_matrix = jitr.InteractionMatrix(1)
interaction_matrix.set_local_interaction(potential, args=(10,))

In [8]:
A = solver.free_matrix(channels["a"][0:1], channels["l"][0:1])
A += solver.interaction_matrix(interaction_matrix, channels[0:1])
A.shape

(40, 40)

In [9]:
a = channels["a"][0:1]

In [10]:
b = solver.precompute_boundaries(a)
x = np.linalg.solve(A, b).reshape(1, nbasis)
x.shape

(1, 40)

In [11]:
R = x @ b.reshape(1, nbasis).T / np.outer(a, a)
R.shape

(1, 1)

In [12]:
Hp = channels["Hp"][0:1]
Hm = channels["Hm"][0:1]
Hpp = channels["Hpp"][0:1]
Hmp = channels["Hmp"][0:1]

In [13]:
Zp = np.diag(Hp) - R * Hpp[:, np.newaxis] * a[:, np.newaxis]
Zm = np.diag(Hm) - R * Hmp[:, np.newaxis] * a[:, np.newaxis]

In [14]:
R.shape

(1, 1)

In [15]:
S = np.linalg.solve(Zp, Zm)
S

array([[-0.8992844+0.43736434j]])

In [16]:
np.absolute(S)

array([[1.]])

In [17]:
R, S, u = solver.solve(interaction_matrix, channels[0:1])

/home/beyerk/Projects/jitr/src/jitr/rmatrix.py:46: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(complex128, 1, 'C', False, aligned=True), Array(complex128, 2, 'A', False, aligned=True))
  R, Ainv = rmatrix_with_inverse(A, b, nchannels, nbasis, a)


In [18]:
S

array([[-0.8992844+0.43736434j]])

In [19]:
R2, S2, u2 = solver.solve(interaction_matrix, channels[1:])

In [20]:
S2

array([[-0.26534903+0.96415242j]])

In [21]:
%%timeit
R, S, u = solver.solve(interaction_matrix, channels[0:1])

294 µs ± 9.54 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [22]:
multi_channel_solver = jitr.RMatrixSolver(nbasis)
multi_channel_interaction = jitr.InteractionMatrix(nchannels)

# set the same interaction in each channel with
# no coupling - should just be 10 copies of the same uncoupled system as before
for i in range(nchannels):
    multi_channel_interaction.set_local_interaction(potential, i, i, args=(10,))

In [23]:
Rm, Sm, xm = multi_channel_solver.solve(multi_channel_interaction, channels)

In [24]:
Sm

array([[-0.8992844 +0.43736434j,  0.        +0.j        ],
       [ 0.        -0.j        , -0.26534903+0.96415242j]])

In [25]:
sys.l

array([0, 1], dtype=int32)

In [26]:
bm = multi_channel_solver.precompute_boundaries(sys.channel_radii)
np.testing.assert_almost_equal(b, bm[:nbasis])
np.testing.assert_almost_equal(b, bm[nbasis : 2 * nbasis])

In [27]:
np.testing.assert_almost_equal(np.linalg.det(Sm.conj().T @ Sm), 1)
np.testing.assert_almost_equal(np.linalg.det(S.conj().T @ S), 1)
np.testing.assert_almost_equal(Sm[1, 0], 0)
np.testing.assert_almost_equal(Sm[0, 1], 0)
np.testing.assert_almost_equal(Rm[1, 0], 0)
np.testing.assert_almost_equal(Rm[0, 1], 0)
np.testing.assert_almost_equal(Sm[1, 1], S2)
np.testing.assert_almost_equal(Rm[1, 1], R2)
np.testing.assert_almost_equal(Sm[0, 0], S)
np.testing.assert_almost_equal(Rm[0, 0], R)

In [28]:
free = multi_channel_solver.free_matrix(sys.channel_radii, sys.l)
interaction = multi_channel_solver.interaction_matrix(
    multi_channel_interaction, channels
)

In [29]:
np.testing.assert_almost_equal(
    solver.free_matrix(sys.channel_radii[0:1], sys.l[0:1]),
    multi_channel_solver.get_channel_block(free, 0, 0),
)

In [30]:
np.testing.assert_almost_equal(
    solver.interaction_matrix(interaction_matrix, channels[0:1]),
    multi_channel_solver.get_channel_block(
        multi_channel_solver.interaction_matrix(multi_channel_interaction, channels),
        0,
        0,
    ),
)

In [31]:
for i in range(nchannels):
    for j in range(nchannels):
        if j != i:
            np.testing.assert_almost_equal(solver.get_channel_block(free, i, j), 0)
            np.testing.assert_almost_equal(
                solver.get_channel_block(interaction, i, j), 0
            )

In [32]:
A = solver.interaction_matrix(interaction_matrix, channels[0:1]) + solver.free_matrix(
    channels["a"][0:1], channels["l"][0:1]
)
Am = multi_channel_solver.interaction_matrix(
    multi_channel_interaction, channels
) + multi_channel_solver.free_matrix(channels["a"], channels["l"])

In [33]:
np.testing.assert_almost_equal(Am[:nbasis, :nbasis], A)
np.testing.assert_almost_equal(bm[:nbasis], b)

In [34]:
x = np.linalg.solve(A, b)
xm = np.linalg.solve(Am, bm)

In [35]:
np.testing.assert_almost_equal(x, xm[:nbasis])

In [36]:
R = x @ b / sys.channel_radii[0] ** 2

In [37]:
Cm = np.linalg.inv(Am)

In [38]:
Cm.shape

(80, 80)

In [39]:
b.T @ Cm[:nbasis, :nbasis] @ b / sys.channel_radii[0] ** 2

(-0.042052875501628596+0j)

In [40]:
bm[:nbasis].T @ Cm[:nbasis, :nbasis] @ bm[:nbasis] / sys.channel_radii[0] ** 2

(-0.042052875501628596+0j)

In [41]:
def Rmatrix(A, b, nchannels, nbasis, a):
    R = np.zeros((nchannels, nchannels), dtype=np.complex128)
    C = np.linalg.inv(A)
    for i in range(nchannels):
        for j in range(nchannels):
            R[i, j] = (
                b[i * nbasis : (i + 1) * nbasis].T
                @ C[i * nbasis : (i + 1) * nbasis, j * nbasis : (j + 1) * nbasis]
                @ b[j * nbasis : (j + 1) * nbasis]
            )
    return R / np.outer(a, a)

In [42]:
Rmatrix(Am, bm, nchannels=nchannels, nbasis=nbasis, a=sys.channel_radii)

array([[-0.04205288+0.j,  0.        +0.j],
       [ 0.        +0.j,  0.08435512+0.j]])